In [60]:
import os
data_dir = 'data3/'
        
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [61]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='sagemaker-ap-southeast-1-876828306900'
data_key = 'balanceddfsmaller.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)



balanceddf=pd.read_csv(data_location)

In [62]:
X=balanceddf["article_text"]
Y=balanceddf.iloc[:,-1]

from sklearn.preprocessing import LabelEncoder

le =LabelEncoder()
y_traint=le.fit_transform(Y)

In [63]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y_traint,test_size=0.2)

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_trainVectorized=vectorizer.fit_transform(X_train)
X_testVectorized=vectorizer.transform(X_test)

In [65]:
X_trainVectorizedTF_array=X_trainVectorized.toarray()
X_testVectorizedTF_array=X_testVectorized.toarray()

In [66]:
y_train.shape

(800,)

In [67]:
X_testVectorizedTF_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
X_trainVectorizedTF_array.shape

(800, 25451)

In [69]:
pd.concat([pd.DataFrame(y_train),pd.DataFrame(X_trainVectorizedTF_array)], axis=1)\
        .to_csv(os.path.join(data_dir, 'train.csv'),header=False, index=False)

In [70]:
pd.DataFrame(X_testVectorizedTF_array).to_csv(os.path.join(data_dir, 'test.csv'),header=False, index=False)

In [72]:
from sagemaker.sklearn.estimator import SKLearn
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [73]:
# should be the name of directory you created to save your features data
data_dir = 'data3/'

# set prefix, a descriptive name for a directory  
prefix = "bias2"

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-ap-southeast-1-876828306900/bias2


In [74]:
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

/cost/20200301-20200401/cost-Manifest.json
/cost/20200301-20200401/dc01c63d-c9ca-4e6f-ac30-ba7d487ffbe8/cost-1.csv.gz
/cost/20200301-20200401/dc01c63d-c9ca-4e6f-ac30-ba7d487ffbe8/cost-Manifest.json
/cost/20200401-20200501/1080bd04-2efa-45d5-8e23-d1ab23de7075/cost-1.csv.gz
/cost/20200401-20200501/1080bd04-2efa-45d5-8e23-d1ab23de7075/cost-Manifest.json
/cost/20200401-20200501/1186c0a3-6c6e-41b3-aa67-cab1aa07e333/cost-1.csv.gz
/cost/20200401-20200501/1186c0a3-6c6e-41b3-aa67-cab1aa07e333/cost-Manifest.json
/cost/20200401-20200501/148828dd-b857-490a-8437-6d227dcf4c8c/cost-1.csv.gz
/cost/20200401-20200501/148828dd-b857-490a-8437-6d227dcf4c8c/cost-Manifest.json
/cost/20200401-20200501/1d9d94fd-b338-49a7-831f-d8a96b13647d/cost-1.csv.gz
/cost/20200401-20200501/1d9d94fd-b338-49a7-831f-d8a96b13647d/cost-Manifest.json
/cost/20200401-20200501/2c05340a-39d3-48ef-a7ec-173e962e66ba/cost-1.csv.gz
/cost/20200401-20200501/2c05340a-39d3-48ef-a7ec-173e962e66ba/cost-Manifest.json
/cost/20200401-20200501/2ce

In [75]:
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [76]:
print(train_location)

s3://sagemaker-ap-southeast-1-876828306900/bias2/train.csv


In [77]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(sagemaker_session.boto_region_name, 'xgboost')

	get_image_uri(region, 'xgboost', '0.90-1').


In [78]:
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(sagemaker_session.default_bucket(), prefix),
                                    sagemaker_session=sagemaker_session)

# And then set the algorithm specific parameters.
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        
                        early_stopping_rounds=10,
                        num_round=300)

In [79]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')

In [80]:
xgb.fit({'train': s3_input_train})

2020-04-27 09:16:46 Starting - Starting the training job...
2020-04-27 09:16:48 Starting - Launching requested ML instances......
2020-04-27 09:18:15 Starting - Preparing the instances for training......
2020-04-27 09:19:10 Downloading - Downloading input data...
2020-04-27 09:19:45 Training - Training image download completed. Training in progress..Arguments: train
[2020-04-27:09:19:46:INFO] Running standalone xgboost training.
[2020-04-27:09:19:46:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-04-27:09:19:46:INFO] File size need to be processed in the node: 81.26mb. Available memory size in the node: 8508.74mb
[2020-04-27:09:19:46:INFO] Determined delimiter of CSV input is ','
[09:19:46] S3DistributionType set as FullyReplicated
[09:19:46] 800x25451 matrix with 20360800 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[09:19:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[0]#01


2020-04-27 09:20:16 Uploading - Uploading generated training model[09:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 14 pruned nodes, max_depth=0
[50]#011train-error:0.03625
[09:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 14 pruned nodes, max_depth=0
[51]#011train-error:0.03625
[09:20:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 12 pruned nodes, max_depth=0
[52]#011train-error:0.03625
[09:20:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[53]#011train-error:0.03625
[09:20:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 14 pruned nodes, max_depth=0
[54]#011train-error:0.03625
[09:20:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 2 pruned nodes, max_depth=5
[55]#011train-error:0.0375
[09:20:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned n

In [81]:
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)

In [82]:
print(test_location)

s3://sagemaker-ap-southeast-1-876828306900/bias2/test.csv


In [83]:
#create transformer object

xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

In [84]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

In [85]:
xgb_transformer.wait()

.....................Arguments: serve
[2020-04-27 09:26:11 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-04-27 09:26:11 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-04-27 09:26:11 +0000] [1] [INFO] Using worker: gevent
[2020-04-27 09:26:11 +0000] [39] [INFO] Booting worker with pid: 39
[2020-04-27 09:26:11 +0000] [40] [INFO] Booting worker with pid: 40
[2020-04-27 09:26:12 +0000] [41] [INFO] Booting worker with pid: 41
[2020-04-27 09:26:12 +0000] [42] [INFO] Booting worker with pid: 42
[2020-04-27:09:26:12:INFO] Model loaded successfully for worker : 40
[2020-04-27:09:26:12:INFO] Model loaded successfully for worker : 39
[2020-04-27:09:26:12:INFO] Model loaded successfully for worker : 41
[2020-04-27:09:26:12:INFO] Model loaded successfully for worker : 42
[2020-04-27:09:26:31:INFO] Sniff delimiter as ','
[2020-04-27:09:26:31:INFO] Determined delimiter of CSV input is ','
[2020-04-27:09:26:31:INFO] Sniff delimiter as ','
[2020-04-27:09:26:31:INFO] Determined delimit

In [86]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-ap-southeast-1-876828306900/xgboost-2020-04-27-09-23-00-018/test.csv.out to data3/test.csv.out


In [87]:

predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [88]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.66

In [89]:
from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.65      0.71      0.68       101
           1       0.67      0.61      0.64        99

   micro avg       0.66      0.66      0.66       200
   macro avg       0.66      0.66      0.66       200
weighted avg       0.66      0.66      0.66       200



<h3>test our model again using the newly deployed endpoint<h3>

In [90]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-------------!

In [91]:

from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [101]:
import numpy as np
def predict(data, rows=50):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_data = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None)

predictions = predict(test_data.as_matrix()[:, 1:])


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [102]:
from sklearn.metrics import accuracy_score
predictions = [round(num) for num in predictions]
accuracy_score(y_test, predictions)

0.495

In [103]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       101
           1       0.49      1.00      0.66        99

   micro avg       0.49      0.49      0.49       200
   macro avg       0.25      0.50      0.33       200
weighted avg       0.25      0.49      0.33       200



/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [104]:
xgb_predictor.delete_endpoint()